# MASK IMAGE GENERATION 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import shutil #recursively moves a file from src to dest and vice versa
import cv2  #resize image , GF
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
from skimage.filters import sobel
from scipy import ndimage as ndi
from skimage import io, color
from skimage.feature import canny

In [4]:
def create_dir(in_dir, out_dir, gdir_1):
    for label in os.listdir(path= in_dir):
        if not os.path.exists(os.path.join(out_dir, gdir_1, label)):
            os.makedirs(os.path.join(out_dir, gdir_1, label))
            
        

In [5]:
def mask_image_process(in_dir = 'input', out_dir= 'processed'):
    gdir_1 = 'mask'
        
    create_dir(in_dir, out_dir, gdir_1)
    count = 0
    for label in os.listdir(path= in_dir):
        print('Processing started for : ',label)
        for image_name in os.listdir(os.path.join(in_dir,label)):
            if not image_name.endswith('.db'):
                in_path = os.path.join(in_dir, label, image_name)
                out_path_1 = os.path.join(out_dir, gdir_1, label, image_name) #path of mask image
               
                #########
                cv_img=cv2.imread(in_path,1)
                cv_img=cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
                ret, bw_img = cv2.threshold(cv_img,120,220,cv2.THRESH_BINARY)
                x = np.where(bw_img == 0, 250,bw_img)
                y =  np.where(x == 255, 0,x)
                z = sobel(bw_img)
                mask_image = ndi.binary_fill_holes(z)             
                mimg = Image.fromarray(np.uint8(mask_image * 255))
                
                mimg.save(out_path_1)
                
                #########
                
               
                count += 1
                #if count >= 100:
                #   break
                
        
        print('------ Processing done ------')

In [6]:
mask_image_process()

Processing started for :  benign
------ Processing done ------
Processing started for :  melanoma
------ Processing done ------


# Mask and Original Images file paths

In [7]:
from skimage.io import imread, imshow

In [8]:
mask_1_path = "processed/mask/benign/"
mask_2_path = "processed/mask/melanoma/"
original_1_input_path = "input/benign/"
original_2_input_path = "input/melanoma/"
extracted_1_images_path = "extracted_images/benign/"
extracted_2_images_path = "extracted_images/melanoma/"

In [9]:
benign_masks = os.listdir(mask_1_path)
melanoma_masks = os.listdir(mask_2_path)
original_benign = os.listdir(original_1_input_path)
original_melanoma = os.listdir(original_2_input_path)

In [10]:
print(len(benign_masks))
print(len(melanoma_masks))
print(len(original_benign))
print(len(original_melanoma))

262
476
262
477


In [12]:
for i in range(0,len(benign_masks)):
    input_image = os.path.join(original_1_input_path, original_benign[i])
    mask_image = os.path.join(mask_1_path, original_benign[i])
    output_image = os.path.join(extracted_1_images_path, original_benign[i])
    img = imread(input_image,0)
    mask = imread(mask_image)
    ret, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    extracted = cv2.bitwise_or(img, img, mask=mask)
    extracted = cv2.cvtColor(extracted, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_image, extracted)

In [13]:
for i in range(0,len(melanoma_masks)):
    input_image = os.path.join(original_2_input_path, original_melanoma[i])
    mask_image = os.path.join(mask_2_path, original_melanoma[i])
    output_image = os.path.join(extracted_2_images_path, original_melanoma[i])
    img = imread(input_image)
    mask = imread(mask_image)
    ret, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    extracted = cv2.bitwise_or(img, img, mask=mask)
    extracted = cv2.cvtColor(extracted, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_image, extracted)